In [1]:
import os
import absl
import urllib
import tempfile

import tensorflow as tf
import tfx

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

tf.get_logger().propogate = False
absl.logging.set_verbosity(absl.logging.INFO)

In [2]:
_data_root = tempfile.mkdtemp()
_data_filepath = os.path.join(_data_root, 'data.csv')
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('/tmp/tmp3rt6zyyq/data.csv', <http.client.HTTPMessage at 0x7f2fe90d03d0>)

In [3]:
context = InteractiveContext()

In [4]:
from tfx.utils.dsl_utils import external_input
from tfx.components import CsvExampleGen

examples = external_input(_data_root)
example_gen = CsvExampleGen(input=examples)
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Processing input csv data /tmp/tmp3rt6zyyq/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-05-15T22_03_35.968184-ef84e95k/CsvExampleGen/examples/1, id: 2)]
        ))

## Using the StatsGen Component

In [6]:
from tfx.components import StatisticsGen

statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'], instance_name='compute-stats')
context.run(statistics_gen)

INFO:absl:Running driver for StatisticsGen.compute-stats
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen.compute-stats
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Generating statistics for split train
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2020-05-15T22_03_35.968184-ef84e95k/StatisticsGen.compute-stats/statistics/2/train.
INFO:absl:Generating statistics for split eval
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2020-05-15T22_03_35.968184-ef84e95k/StatisticsGen.compute-stats/statistics/2/eval.
/opt/conda/lib/python3.7/site-packages/tensorflow_data_validation/arrow/arrow_util.py:239: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):
INFO:absl:Running publisher for StatisticsGen.compute-stats
INFO:absl:MetadataStore 

ExecutionResult(
    component_id: StatisticsGen.compute-stats
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: /tmp/tfx-interactive-2020-05-15T22_03_35.968184-ef84e95k/StatisticsGen.compute-stats/statistics/2, id: 3)]
        ))

In [7]:
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


## Using the StatsGen Component With a Schema

Schema in TFX is an instance of the TensorFlow Metadata [Schema proto](https://github.com/tensorflow/metadata/blob/master/tensorflow_metadata/proto/v0/schema.proto)

In [ ]:
from tfx.components import ImporterNode
from tfx.types import standard_artifacts

# In this setting, you will invoke StatisticsGen with a curated schema that has been 
# imported by an ImporterNode like this:
user_schema_importer = ImporterNode(
    instance_name='import_user_schema',
    source_uri=..., # directory containing only schema text proto
    artifact_type=standard_artifacts.Schema
)

compute_eval_stats = components.StatisticsGen(
    examples=example_gen.outputs['examples'],
    schema=user_schema_importer.outputs['result'],
    name='compute-stats'
)